In [3]:
import yfinance as yf
import os
import pandas as pd
from datetime import datetime, date, timedelta
from dateutil.parser import parse as dtparse
from concurrent.futures import ThreadPoolExecutor, as_completed

# ------------------ USER SETTINGS ------------------
MAX_EXPIRIES_PER_TICKER = 8     # to avoid rate-limit pain (nearest 7 + Jan'26 if present)
LAST_PRICE_MAX = 1.5            # <= $1.50 contracts only
VOL_MIN = 300                   # minimum volume to be considered
VOL_OI_MIN = 2.0                # volume/open interest threshold
END_DATE_CUTOFF = "2026-01-31"  # scan expiries up to end-Jan 2026

base_date = datetime.now().strftime('%Y-%m-%d')
prefix = f"unusual_options_scan_{base_date}"
ext = ".csv"

# Find the highest existing index for today (to avoid overwriting multiple runs per day)
existing_indices = []
for fname in os.listdir('.'):
    if fname.startswith(prefix) and fname.endswith(ext):
        try:
            num = int(fname[len(prefix)+1:-len(ext)])
            existing_indices.append(num)
        except ValueError:
            pass

# Determine next index
next_index = max(existing_indices) + 1 if existing_indices else 1
SAVE_CSV = f"{prefix}_{next_index}{ext}"

print(f"Next file to save: {SAVE_CSV}")

# ---------------------------------------------------
# NASDAQ-100 tickers (plus extras; harmless if a few changed)
NASDAQ100 = [
    "AAPL","MSFT","NVDA","AMZN","META","GOOGL","GOOG","TSLA","AVGO","COST","AFRM",
    "NFLX","PEP","ADBE","AMD","LIN","TMUS","CSCO","QCOM","TXN","AMAT",
    "INTU","HON","INTC","BKNG","SBUX","MU","AMGN","PDD","REGN","LRCX",
    "ADP","ISRG","ABNB","MDLZ","VRTX","ASML","GILD","ADI","PANW","KLAC",
    "PYPL","CRWD","CSX","WDAY","CHTR","MAR","NXPI","ROP","AEP","KDP",
    "MELI","FTNT","ORLY","SNPS","CDNS","MNST","CTAS","DXCM","PCAR","LULU",
    "MRVL","MCHP","ROST","EXC","ODFL","ADSK","ATVI","IDXX","EA",
    "PAYX","CTSH","TEAM","XEL","WDAY","DDOG","ZS","SPLK","BKR","ALGN",
    "AZN","CEG","VRSK","SIRI","PDD","LCID","RIVN","BIDU","JD","BMRN",
    "DOCU","VRSN","NTES","MRNA","ANSS","CSGP","CHKP","MTCH","CRWD","OKTA",
    "NEE","JNJ","SMCI","STZ","TMQ","PLTR","XYZ","HOOD","ORCL","UPST",
    "TSM","SHOP","SPOT","LLY","HIMS","UNH","DELL","COIN","OSCR","SNOW",
    "QUBT","RGTI","CRWV","RKLB","BA","QCOM","PANW","JPM","GS","BABA","BIDU","USAR","ONON","VIX","OKLO",
    "QS","CRML","MP","QBTS","JEF","GKOS","GSK","AMGN","ROKU","RH","FCX","DASH","CHWY","CCJ","FI","TEAM",
    "SBET","METC","AVAV","MTSR","NTLA","ALAB","ALK","PINS","TEM","AZN","CE","WWW","TREX","LVS","SNDK","BBAI",
    "NNN","QURE","LENZ","A","SYM","KSS","EXEL","MDB"
    # "QQQ","SPXW","SPY"  # left commented intentionally
]
# Deduplicate (list may contain a couple repeats above)
TICKERS = sorted(list(dict.fromkeys(NASDAQ100)))

# ------------------ Helpers: stock volume trends ------------------
def _pct_change_block(vol_series, k):
    """
    Percent change of sum(last k trading days) vs sum(previous k days).
    Returns float or None if insufficient data or prior-sum == 0.
    """
    vol = pd.Series(vol_series).dropna().astype(float)
    if len(vol) < 2 * k:
        return None
    recent = vol.iloc[-k:].sum()
    prior = vol.iloc[-2*k:-k].sum()
    if prior == 0:
        return None
    return (recent - prior) / prior

def get_stock_volume_trend(ticker, period="90d"):
    """
    Pull underlying daily volume and compute:
      - vol_1w_inc: 5d vs prior 5d
      - vol_2w_inc: 10d vs prior 10d
    """
    try:
        hist = yf.Ticker(ticker).history(period=period, interval="1d", auto_adjust=False)
        if hist.empty or "Volume" not in hist.columns:
            return {"ticker": ticker, "vol_1w_inc": None, "vol_2w_inc": None}
        vol = hist["Volume"]
        return {
            "ticker": ticker,
            "vol_1w_inc": _pct_change_block(vol, 5),
            "vol_2w_inc": _pct_change_block(vol, 10),
        }
    except Exception:
        return {"ticker": ticker, "vol_1w_inc": None, "vol_2w_inc": None}

# ------------------ Options scanning ------------------
def safe_option_chain(tkr, exp):
    """Return (calls, puts) DataFrames or (None, None) on failure."""
    try:
        oc = tkr.option_chain(exp)
        c = oc.calls.copy()
        p = oc.puts.copy()
        c["type"] = "CALL"
        p["type"] = "PUT"
        for df in (c, p):
            df["expiration"] = exp
        return c, p
    except Exception:
        return None, None

def pick_expiries(all_exps):
    """
    Choose a practical subset:
    - nearest expiries in order, up to MAX_EXPIRIES_PER_TICKER - 1
    - plus Jan 2026 (any Jan-2026 date if present)
    - only expiries <= END_DATE_CUTOFF and >= today
    """
    OFFSET_DAYS = 0
    today_offset = (date.today() + timedelta(days=OFFSET_DAYS)).isoformat()
    cutoff = END_DATE_CUTOFF
    exps = [e for e in all_exps if today_offset <= e <= cutoff]
    exps_sorted = sorted(exps)
    chosen = exps_sorted[:max(0, MAX_EXPIRIES_PER_TICKER - 1)]
    jan26 = [e for e in exps_sorted if e.startswith("2026-01")]
    if jan26:
        jan_pick = jan26[0]
        if jan_pick not in chosen:
            chosen.append(jan_pick)
    return chosen

def scan_ticker(ticker):
    tkr = yf.Ticker(ticker)
    try:
        all_exps = tkr.options
    except Exception:
        return pd.DataFrame()

    if not all_exps:
        return pd.DataFrame()

    exps = pick_expiries(all_exps)
    rows = []
    for exp in exps:
        calls, puts = safe_option_chain(tkr, exp)
        if calls is None:
            continue
        df = pd.concat([calls, puts], ignore_index=True)

        # Clean columns (Yahoo schema can vary slightly)
        for col in ["lastPrice","volume","openInterest","strike"]:
            if col not in df.columns:
                df[col] = 0

        # Filter rules:
        # 1) lastPrice <= $1.50
        # 2) volume >= VOL_MIN
        # 3) vol/openInterest >= VOL_OI_MIN (unusual-ish)
        df["vol_oi"] = df["volume"] / df["openInterest"].replace(0, 1)
        flt = (
            (df["lastPrice"] <= LAST_PRICE_MAX) &
            (df["volume"] >= VOL_MIN) &
            (df["vol_oi"] >= VOL_OI_MIN)
        )
        df = df.loc[flt, ["contractSymbol","type","strike","lastPrice","volume",
                          "openInterest","vol_oi","expiration"]]
        df["ticker"] = ticker

        # Simple score to rank (volume * vol/oi)
        df["score"] = df["volume"] * df["vol_oi"]
        rows.append(df)

    if not rows:
        return pd.DataFrame()
    out = pd.concat(rows, ignore_index=True)
    return out

# ------------------ Main ------------------
def main():
    all_hits = []

    # Tune max_workers depending on network / rate-limit tolerance
    max_workers = min(20, len(TICKERS))  # 15–25 is usually a sweet spot

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_ticker = {executor.submit(scan_ticker, tk): tk for tk in TICKERS}

        for i, future in enumerate(as_completed(future_to_ticker), 1):
            tk = future_to_ticker[future]
            try:
                hits = future.result()
                if not hits.empty:
                    all_hits.append(hits)
                    print(f"[{i:3d}/{len(TICKERS)}] ✅ {tk} — found {len(hits)} matches")
                else:
                    print(f"[{i:3d}/{len(TICKERS)}] {tk} — no matches")
            except Exception as e:
                print(f"[{i:3d}/{len(TICKERS)}] {tk} — ❌ error: {e}")

    if not all_hits:
        print("No matches found with current filters. Consider lowering VOL_MIN or VOL_OI_MIN.")
        return

    df = pd.concat(all_hits, ignore_index=True)

    # Rank per-ticker by max score, keep top 10 tickers, then show their top rows
    per_ticker = (
        df.groupby("ticker")["score"]
          .max()
          .reset_index()
          .sort_values("score", ascending=False)
    )
    top10_tickers = per_ticker["ticker"].head(10).tolist()

    final = (
        df[df["ticker"].isin(top10_tickers)]
        .sort_values(["ticker","score"], ascending=[True, False])
        .copy()
    )

    # ---- NEW: add underlying stock volume trend columns (per ticker) ----
    uniq_tickers = final["ticker"].dropna().unique().tolist()
    # Sequential fetch to be gentle on Yahoo; could be parallelized if desired
    vol_trends = [get_stock_volume_trend(tk, period="90d") for tk in uniq_tickers]
    trend_df = pd.DataFrame(vol_trends)

    final = final.merge(trend_df, on="ticker", how="left")

    # Save CSV with new columns
    final_cols = [
        "ticker","type","strike","lastPrice","volume","openInterest",
        "vol_oi","expiration","contractSymbol","score","vol_1w_inc","vol_2w_inc"
    ]
    final[final_cols].to_csv(SAVE_CSV, index=False)
    print(f"\nSaved {len(final)} matches to: {SAVE_CSV}")

    # Pretty console output
    def _fmt_pct(x):
        return "—" if pd.isna(x) else f"{x*100:.1f}%"

    print("\n=== Underlying volume trend (per ticker in final) ===")
    trend_print = trend_df.copy()
    if not trend_print.empty:
        trend_print = trend_print.sort_values(
            ["vol_1w_inc","vol_2w_inc"], ascending=[False, False]
        )
        for _, r in trend_print.iterrows():
            print(f"{r['ticker']:>6}  1w: {_fmt_pct(r['vol_1w_inc'])}   2w: {_fmt_pct(r['vol_2w_inc'])}")
    else:
        print("No trend data available.")

    print("\n=== Top 10 tickers (by max score) ===")
    for _, row in per_ticker[per_ticker["ticker"].isin(top10_tickers)].iterrows():
        print(f"{row['ticker']:>5}  score={row['score']:.1f}")

    print("\n=== Sample rows (with volume trend columns) ===")
    print(final[final_cols].head(25).to_string(index=False))

if __name__ == "__main__":
    main()


Next file to save: unusual_options_scan_2025-12-01_10.csv
[  1/166] ANSS — no matches
[  2/166] ATVI — no matches
[  3/166] ALK — no matches
[  4/166] AEP — no matches
[  5/166] A — no matches
[  6/166] AZN — no matches
[  7/166] ADI — no matches
[  8/166] ✅ AMGN — found 1 matches
[  9/166] ADSK — no matches
[ 10/166] ABNB — no matches
[ 11/166] ✅ AFRM — found 4 matches
[ 12/166] ✅ ASML — found 2 matches
[ 13/166] ✅ AMD — found 9 matches
[ 14/166] ALGN — no matches
[ 15/166] ADP — no matches
[ 16/166] ✅ ADBE — found 5 matches
[ 17/166] ✅ AMZN — found 4 matches
[ 18/166] ✅ ALAB — found 4 matches
[ 19/166] ✅ AAPL — found 11 matches
[ 20/166] AVAV — no matches
[ 21/166] ✅ AMAT — found 4 matches
[ 22/166] ✅ BABA — found 3 matches
[ 23/166] ✅ AVGO — found 10 matches
[ 24/166] BA — no matches
[ 25/166] BMRN — no matches
[ 26/166] BBAI — no matches
[ 27/166] BKR — no matches
[ 28/166] CHKP — no matches
[ 29/166] CE — no matches
[ 30/166] CTSH — no matches
[ 31/166] CSGP — no matches
[ 32/166]